In [1]:
import os, time
os.environ["MUJOCO_GL"] = "egl"
os.environ["EGL_PLATFORM"] = "surfaceless"
os.environ["LD_LIBRARY_PATH"] = "/usr/lib/nvidia:" + os.environ.get("LD_LIBRARY_PATH", "")

In [11]:
import gymnasium as gym
import numpy as np

env = gym.make("kitchen_relax-v1")
obs = env.reset()

print("=== Spaces ===")
print("action_space:", env.action_space)
print("obs type:", type(obs), "obs shape:", getattr(obs, "shape", None))

# 取出 mujoco sim（不同封装可能叫 env.sim 或 env.env.sim）
sim = getattr(env, "sim", None)
if sim is None and hasattr(env, "env"):
    sim = getattr(env.env, "sim", None)

print("\n=== MuJoCo model/data ===")
print("sim exists:", sim is not None)
if sim is not None:
    model, data = sim.model, sim.data
    print("nu (actuators):", model.nu)
    print("nq (qpos):", model.nq, "nv (qvel):", model.nv)

    # actuator / joint 名称（如果可用）
    try:
        print("\nActuators:")
        for i in range(model.nu):
            print(i, model.actuator_id2name(i))
    except Exception as e:
        print("actuator_id2name not available:", e)

    try:
        print("\nJoints (first 20):")
        for i in range(min(model.njnt, 20)):
            print(i, model.joint_id2name(i))
    except Exception as e:
        print("joint_id2name not available:", e)

    # 当前状态向量
    print("\nqpos[:15]:", data.qpos[:15])
    print("qvel[:15]:", data.qvel[:15])
    print("ctrl[:]:", data.ctrl[:])

env.close()


NameNotFound: Environment `kitchen_relax` doesn't exist.

In [6]:
import gymnasium as gym
import gymnasium_robotics
from gymnasium.wrappers import RecordVideo
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

gym.register_envs(gymnasium_robotics)

save_dir = f"/home/xli990/paichichi/videos/franka/reach_2views/{int(time.time())}"

env = gym.make(
    "FrankaKitchen-v1",
    tasks_to_complete=["microwave", "kettle"],
    render_mode="rgb_array",
)

# 录制：每个 episode 都录（你也可以改成只录第一个）
env = RecordVideo(
    env,
    video_folder=save_dir,
    episode_trigger=lambda episode_id: True,
    name_prefix="franka_kitchen",
)

obs, info = env.reset(seed=42)
env.action_space.seed(42)

try:
    for t in range(50):
        a = env.action_space.sample()
        obs, r, term, trunc, info = env.step(a)

        d_micro = np.linalg.norm(obs["achieved_goal"]["microwave"] - obs["desired_goal"]["microwave"])
        d_kettle = np.linalg.norm(obs["achieved_goal"]["kettle"] - obs["desired_goal"]["kettle"])

        print(t, "r=", r, "d_micro=", float(d_micro), "d_kettle=", float(d_kettle),
              "done=", info["episode_task_completions"])
        if term or trunc:
            break
finally:
    env.close()

print("Videos saved to:", save_dir)

Videos saved to: /home/xli990/paichichi/videos/franka/reach_2views/1769945230


In [9]:
trunc

False